In [1]:
import torch
import numpy as np
import torch.nn.functional as F
import random
from sklearn.datasets import load_iris
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from math import*

In [2]:
#此處 M,D指係數矩陣
def legendre(x,M):
    return torch.matmul(M,torch.Tensor([[1],[x],[0.5*(3*(x**2)-1)],[0.5*(5*(x**3)-3*x)]]))
def dlegendre(x,D):
    return torch.matmul(D,torch.Tensor([[0],[1],[3*x],[0.5*(15*(x**2)-3)]]))

In [3]:
X,y = load_iris(return_X_y=True)
y = y[:,np.newaxis]

X = Variable(torch.Tensor(X))
y = Variable(torch.LongTensor(y))

dataset = torch.cat((X,y),1)
dataset = DataLoader(dataset,shuffle=True)

X = []
y = []
for i in dataset:
    temp = i.numpy().squeeze()
    X.append(temp[0:4])
    y.append(int(temp[4]))
X = np.array(X)
y = np.array(y)

X_train = Variable(torch.Tensor(X[:125,:]))
y_train = Variable(torch.LongTensor(y[:125]))

X_test = Variable(torch.Tensor(X[125:,:]))
y_test = Variable(torch.LongTensor(y[125:]))
#X共150筆 4個feature

In [4]:
class ODE_block(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.h1=torch.nn.Linear(4,4)
        self.h2=torch.nn.Linear(4,4)
        self.h3=torch.nn.Linear(4,4)
        self.h4=torch.nn.Linear(4,4)
    def forward(self,x):
        x1=F.relu(self.h1(x))
        x2=F.relu(self.h2(x1))
        x3=F.relu(self.h3(x2))
        x4=F.relu(self.h4(x3))
        self.x1=x1
        self.x2=x2
        self.x3=x3
        self.x4=x4
        return x4
up_layer = [torch.nn.Linear(4,4)]
middle_layer = [ODE_block()]
down_layer = [torch.nn.Linear(4,3),torch.nn.LogSoftmax()]
net = torch.nn.Sequential(*up_layer,*middle_layer,*down_layer)
loss_func = torch.nn.NLLLoss()
print(net)

Sequential(
  (0): Linear(in_features=4, out_features=4, bias=True)
  (1): ODE_block(
    (h1): Linear(in_features=4, out_features=4, bias=True)
    (h2): Linear(in_features=4, out_features=4, bias=True)
    (h3): Linear(in_features=4, out_features=4, bias=True)
    (h4): Linear(in_features=4, out_features=4, bias=True)
  )
  (2): Linear(in_features=4, out_features=3, bias=True)
  (3): LogSoftmax(dim=None)
)


In [5]:
#train ODE BLOCK
t = torch.Tensor([-1*(sqrt(525+70*sqrt(30)))/(35),-1*(sqrt(525-70*sqrt(30)))/(35),(sqrt(525-70*sqrt(30)))/(35),(sqrt(525+70*sqrt(30)))/(35)])
M = Variable(torch.Tensor(0.5*np.random.randn(4,4)),requires_grad=True)
aopt = torch.optim.Adam(net[1].parameters(),lr=0.001)
gama = 0.4
lr = 0.001
epoch = 100

for itr in range(epoch):
    pick = random.randrange(0,len(X_train))
    x = X_train[pick]
    pd = net(x)
    loss1 = gama*loss_func(pd.unsqueeze(0),y_train[pick].unsqueeze(0))#gama為超參數
    loss2 = 0
    tp = [net[1].x1,net[1].x2,net[1].x3,net[1].x4]
    for j in range(len(t)):
        if j == 0 or j == 3:
            loss2 += ((18-sqrt(30))/36)*torch.norm(dlegendre(t[j],M)-tp[j])
        else:
            loss2 += ((18+sqrt(30))/36)*torch.norm(dlegendre(t[j],M)-tp[j])
    total_loss = loss1+loss2
    if total_loss < 1e-5:
        print("Finished...")
        break
    total_loss.backward(retain_graph = True)
    M.data.zero_()
    M.data.sub_(lr*M.grad.data)
    
    aopt.zero_grad()
    loss2.backward()
    aopt.step()
    
print("Training completed...")

c:\programdata\miniconda3\lib\site-packages\torch\nn\modules\container.py:119: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Training completed...


In [6]:
print(total_loss,net[1].x1,net[1].x2,net[1].x3,net[1].x4,sep='\n')

tensor(1.5234, grad_fn=<AddBackward0>)
tensor([0.3937, 0.8338, 0.1498, 0.3305], grad_fn=<ReluBackward0>)
tensor([0.0570, 0.0000, 0.0000, 0.3730], grad_fn=<ReluBackward0>)
tensor([0.0000, 0.0000, 0.0679, 0.0000], grad_fn=<ReluBackward0>)
tensor([0., 0., 0., 0.], grad_fn=<ReluBackward0>)


In [7]:
#train up and down layer
aopt1 = torch.optim.Adam(net[0].parameters(),lr=0.001)
aopt2 = torch.optim.Adam(net[2].parameters(),lr=0.001)
for i in range(500):
    pd = net(X_train)
    loss = loss_func(pd,y_train)
    aopt1.zero_grad()
    aopt2.zero_grad()
    loss.backward()
    aopt1.step()
    aopt2.step()
    
print("Training completed...")

Training completed...


In [8]:
def class_predict(x):
    l = []
    temp = net(x)
    for i in temp:
        i=list(i)
        l.append(i.index(max(i)))
    return l

def cal_accuracy(yp,yt):
    ans = list(np.array(yp)-np.array(yt))
    print("accuracy:%.2f"%(ans.count(0)/len(yt)))

In [9]:
l=class_predict(X_test)
print(l)
cal_accuracy(l,y_test)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
accuracy:0.32
